In [87]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#import csv
pd.set_option('display.max_rows', None)  # Display all rows


In [88]:
#if not os.path.exists("images"):
#    os.mkdir("images")

dataset = "rfid"

def process_file(file_number, date):
    folder_path = dataset + '_' + date + '/'
    file_path = os.path.join(folder_path, f'rfid_{date}_{file_number}.csv')
    if os.path.exists(file_path):
        #print(file_path)
        if file_number == "final":
            #file_path = os.path.join("rfid_fabrica", f'rfid_final.csv')
            df = pd.read_csv(file_path, parse_dates=['Stamp'])
            if df.shape[1] == 6:
                df = df.drop(df.columns[0], axis=1)
                df = df.drop(df.columns[1], axis=1)
                df = df.drop(df.columns[2], axis=1)


            if df.shape[1] == 4:
                df = df.drop(df.columns[3], axis=1)
                df = df.drop(df.columns[1], axis=1)
            df.to_csv(file_path, index = False)

        else:
            df = pd.read_csv(file_path, parse_dates=['ts'])

            # Remove first empty column if it exists
            if df.shape[1] == 3:
                df = df.drop(df.columns[0], axis=1)
            # Remove entries with 'gabarit' equal to 0
            df= df[df['gabarit'] != 0]

            # Write the updated DataFrame back to the CSV file
            df.to_csv(file_path, index = False)
            # Reset the index of the DataFrame
            df = df.reset_index(drop = True) 
    else:
        return pd.DataFrame

    return df

day = "27"
month = "09"
date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day

df1 = process_file("1",date)
df2 = process_file("2",date)
df3 = process_file("3",date)
df4 = process_file("final",date)




In [89]:
# calculates time and writes it in dataframe
def calc_time_in_station(df, df_in, index_i, index_f, cnt):
    if index_f - index_i == 0:
        time_in_station = 1
    else: 
        time_in_station = ((df.loc[index_f, 'ts'] - df.loc[index_i, 'ts']).total_seconds() + 1)

    df_in.loc[cnt,'ts'] = df.loc[index_i, 'ts']
    df_in.loc[cnt,'gabarit'] = df.loc[index_i, 'gabarit']
    df_in.loc[cnt,'time_in_station'] = time_in_station

    cnt += 1

    return df_in, cnt


def time_in_station_fun(df):
    """
    Calculates and records the time spent in a station for each occurrence of a 'gabarit' in a DataFrame in seconds.

    Parameters:
        - df: DataFrame containing timestamps and 'gabarit' values.

    Returns:
        DataFrame with an added 'time_in_station' column representing the time
        spent in the station for each occurrence (in seconds).
    """
    
    curr_gabarit = 0
    index_i = 0  # Initialize index_i outside the loop
    index_f = 0  # Initialize index_f outside the loop
    cnt = 0
    df_in = pd.DataFrame(columns=['ts', 'gabarit', 'time_in_station'])
    #df_in['ts'] = pd.to_datetime(df['ts'])

    for i in range(df.shape[0]): #0 to 129
        if i != 0: # not first index
            if ((df.loc[i, 'gabarit'] != curr_gabarit)):
                # routine to present final result
                df_in, cnt = calc_time_in_station(df, df_in, index_i, index_f, cnt)
            
          
            # new gabarit is found, i = f
            if df.loc[i, 'gabarit'] != df.loc[i-1, 'gabarit']:
                curr_gabarit = df.loc[i, 'gabarit']
                index_i = i # starts the interval
                index_f = i
        else: # for first entry, i = f
            curr_gabarit = df.loc[i, 'gabarit']
            index_i = i
            index_f = i

        
        if i != df.shape[0] - 1: # not last index 
            if df.loc[i, 'gabarit'] == df.loc[i+1, 'gabarit']:
                index_f = i+1
        else:
            index_f = i           
            df_in, cnt = calc_time_in_station(df, df_in, index_i, index_f, cnt)

    return df_in


def plot_time_in_station(df, station):
    df = df[df['time_in_station'] != 0]
    # Create a scatter plot using Plotly Express
    if station == 1:
        title_plot_var = 'Begginning of Stage 3'
    elif station == 2:
        title_plot_var = 'End of Stage 3'
    elif station == 3:
        title_plot_var = 'Begginning of Stage 7'

    title_plot = 'Plot of Time in ' + title_plot_var + ' over Time'

    ts_values = np.array(df['ts'])
    df['time_in_station'] = df['time_in_station'] / 60

    # Create a scatter plot using go.Scatter
    scatter = go.Scatter(x=ts_values, y=df['time_in_station'], mode='markers', 
                        name='Scatter Plot', marker=dict(color='blue'))
    


    # Create a layout
    layout = go.Layout(
        title=title_plot,
        xaxis=dict(title='Timestamp', range=[date_year + ' 09:00:00', date_year + ' 18:00:00']),  # Set the range for the x-axis
        yaxis=dict(title='Time in Station', range=[-0.2, 3])  # Set the range for the y-axis
    )

    # Create a figure and add the scatter trace
    fig = go.Figure(data=[scatter], layout=layout)

    # Show the plot
    fig.show()
    fig.write_image("images/fig_" + dataset + "_" + day + "_" + month + "_" + str(station) + ".svg")



In [90]:
plot = 1
display_df = 0
df_in_1 = time_in_station_fun(df1)
df_in_2 = time_in_station_fun(df2)
df_in_3 = time_in_station_fun(df3)

if display_df:
    display(df_in_1)
    display(df_in_2)
    display(df_in_3)

if plot:
    plot_time_in_station(df_in_1,1)
    plot_time_in_station(df_in_2,2)
    plot_time_in_station(df_in_3,3)



In [91]:


def time_between_station(df1, df2, station1, station2, max_time):
    """
    Finds and records the time difference between occurrences of the same 'gabarit' in two DataFrames,
    df1 and df2, within a specified time window.

    Parameters:
        - df1: DataFrame containing timestamps and 'gabarit' values for station1.
        - df2: DataFrame containing timestamps and 'gabarit' values for station2.
        - station1: Identifier for the first station.
        - station2: Identifier for the second station.
        - max_time: Maximum time window to search for matching occurrences (in minutes).
    
    Returns:
        DataFrame with columns 'gabarit', 'ts' (timestamp), and 'time_bet_station1_station2'
        representing the time difference (in minutes) between corresponding occurrences in
        station1 and station2 within the specified time window.
    """
    
    df_bet = pd.DataFrame(columns=['gabarit', f'time_bet_{station1}_{station2}'])
    df_bet['ts'] = pd.NaT  # initialize as datetime type

 
    if (station1 != 3 and station2 != 4): ##1 2 or 2 3 
        column_order = ['ts', 'gabarit', f'time_bet_{station1}_{station2}']
        df_bet = df_bet[column_order]
        df_bet = find_matching_gabarit(df1, df2, station1, station2, max_time, df_bet)
    else:
        df_bet['model'] = ''
        df_bet['state'] = ''
        column_order = ['ts', 'gabarit', 'model', f'time_bet_{station1}_{station2}','state']
        df_bet = df_bet[column_order]
        df_bet = find_matching_gabarit_final(df1, df2, df_bet)

    return df_bet


def find_matching_gabarit(df1,df2,station1, station2, max_time, df_bet):

    cnt = 0
            
    for index in range(df1.shape[0]):
        start_time = df1.loc[index, 'ts']
        end_time = start_time + pd.Timedelta(minutes=max_time)
        matching_rows = df2[(df2['ts'] >= start_time) & 
                            (df2['ts'] <= end_time) 
                                & (df2['gabarit'] == df1.loc[index,'gabarit'])]
        if not matching_rows.empty:
            first_occurrence = matching_rows.iloc[0]
            
            # df1: initial time stamp, time_in_station_1
            # df2: initial time stamp, time_in_station_2
            if (station1 == 1 and station2 == 2):
                time_bet = (first_occurrence['time_in_station'] +
                        (first_occurrence['ts'] - df1.loc[index, 'ts']).total_seconds()) # in seconds
            else:
                time_bet = ((first_occurrence['ts'] - df1.loc[index, 'ts'] # in seconds
                            - pd.Timedelta(seconds = df1.loc[index, 'time_in_station'])).total_seconds())

                #df1: 10:06:00 17
                #     10:07:30 17
                #df2: 10:07:10 17
            #    time_bet = (df1.loc[index, 'time_in_station'] + first_occurrence['time_in_station'])

            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
            df_bet.loc[cnt, f'time_bet_{station1}_{station2}'] = time_bet #round,2
            cnt += 1
        
    return df_bet

def find_matching_gabarit_final(df1, df2, df_bet):

    cnt = 0
            
    for index in range(df1.shape[0]):
        start_time = (df1.loc[index, 'ts'] + pd.Timedelta(seconds = df1.loc[index, 'time_in_station'])
                      + pd.Timedelta(seconds=20)) # not possible to have a test that takes less than 20 seconds
        matching_rows = df2[(df2['Stamp'] >= start_time)]


        if not matching_rows.empty:
            first_occurrence = matching_rows.iloc[0]
            
            # df1: initial time stamp, time_in_station_1
            # df2: initial time stamp, time_in_station_2
            time_bet = ((first_occurrence['Stamp'] - df1.loc[index, 'ts']).total_seconds()) # in minutes

            df_bet.loc[cnt, 'ts'] = df1.loc[index, 'ts']  # Copy the timestamp for reference
            df_bet.loc[cnt, 'gabarit'] = df1.loc[index,'gabarit']
            #print(cnt)
            #print(type(first_occurrence['Model']))
            #print(first_occurrence['Model'])
            df_bet.loc[cnt, 'model'] = str(first_occurrence['Model'])
            df_bet.loc[cnt, 'state'] = str(first_occurrence['Status'])
            df_bet.loc[cnt, f'time_bet_3_4'] = time_bet
            cnt += 1
        
    return df_bet



display_df_ = 0

df_bet_12 = time_between_station(df_in_1, df_in_2, 1, 2, max_time = 5)
df_bet_23 = time_between_station(df_in_2, df_in_3, 2, 3, max_time = 20)
df_bet_34 = time_between_station(df_in_3, df4, 3, 4, max_time = 0)

if display_df_:
    display(df_bet_12)
    display(df_bet_23)
    display(df_bet_34)




In [92]:
def perc_of_gabarits_found(df_in, df4, station):
    gab_scanned = df_in.shape[0]
    gab_factory = df4.shape[0]
    perc = gab_scanned / gab_factory * 100
    perc = round(perc,2)
    
    if station == 4:
        #display(df4)
        #display(df_in) 
        ok_count = df_in['state'].value_counts().get('OK', 0)
        #print(f'ok: {ok_count}')
        ko_count =gab_scanned - ok_count
        perc_ok = round(ok_count / gab_scanned *100)
        perc_ko = round(100 - perc_ok)
        print(f"Percentage of gabarits scanned in every sensor: {perc}%, which of those occured OK:{ok_count} and KO:{ko_count}")
    else: 
        print(f"Percentage of gabarits scanned in sensor {station}: {perc}%")
    
    return perc


perc1 = perc_of_gabarits_found(df_in_1, df4, 1)
perc2 = perc_of_gabarits_found(df_in_2, df4, 2)
perc3 = perc_of_gabarits_found(df_in_3, df4, 3)


Percentage of gabarits scanned in sensor 1: 80.78%
Percentage of gabarits scanned in sensor 2: 22.82%
Percentage of gabarits scanned in sensor 3: 48.05%


In [93]:

def all_time_between_station(df12, df23, df34):
    #df_bet_all = pd.DataFrame(columns = ['ts1','time_bet_12','ts2', 'ts2','time_bet_23','ts3','time_bet_34','gabarit','model'])
    df_bet_all = pd.DataFrame(columns = ['ts','gabarit', 'model','time_bet_1_2','time_bet_2_3','time_bet_3_4','state']) 
    #display(df12)
    #display(df23)
    #display(df34)

    cnt = 0
    for index in range(df12.shape[0]):

        start_time = (df12.loc[index, 'ts'] + pd.Timedelta(seconds = df12.loc[index, f'time_bet_1_2'])
                       - pd.Timedelta(seconds=2))
        end_time = start_time + pd.Timedelta(seconds=3)

        matching_rows = df23[(df23['ts'] >= start_time) & (df23['ts'] <= end_time) 
                                & (df23['gabarit'] == df12.loc[index,'gabarit'])]
        if not matching_rows.empty:
            
            first_occurrence = matching_rows.iloc[0]

            #df_bet_all.loc[cnt, 'ts'] = df12.loc[index, 'ts']  # Copy the timestamp for reference
            #df_bet_all.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
            #df_bet_all.loc[cnt, 'time_bet_1_2'] = df12.loc[index, 'time_bet_1_2'] 
            #df_bet_all.loc[cnt, 'time_bet_2_3'] = first_occurrence['time_bet_2_3'] 
            #cnt += 1

            start_time2 = (first_occurrence['ts'] + pd.Timedelta(seconds = first_occurrence[f'time_bet_2_3']) 
                          - pd.Timedelta(seconds=1))
            end_time2 = start_time2 +   pd.Timedelta(seconds=3)
            matching_rows2 = df34[(df34['ts'] >= start_time2) & (df34['ts'] <= end_time2) 
                                & (df34['gabarit'] == first_occurrence['gabarit'])]
            
            if not matching_rows2.empty:

                first_occurrence2 = matching_rows2.iloc[0]
                df_bet_all.loc[cnt, 'ts'] = df12.loc[index, 'ts']  # Copy the timestamp for reference
                df_bet_all.loc[cnt, 'gabarit'] = first_occurrence['gabarit']
                df_bet_all.loc[cnt, 'time_bet_1_2'] = df12.loc[index, 'time_bet_1_2'] 
                df_bet_all.loc[cnt, 'time_bet_2_3'] = first_occurrence['time_bet_2_3'] 

                df_bet_all.loc[cnt, 'time_bet_3_4'] = first_occurrence2['time_bet_3_4']
                df_bet_all.loc[cnt, 'model'] = str(first_occurrence2['model'])
                df_bet_all.loc[cnt, 'state'] = str(first_occurrence2['state'])
            
                cnt += 1

    return df_bet_all



def plot_time_in_station(df):
    #display(px.data.medals_wide())

    df['time_bet_1_2'] = df['time_bet_1_2'].astype(int)
    df['time_bet_2_3'] = df['time_bet_2_3'].astype(int)
    df['time_bet_3_4'] = df['time_bet_3_4'].astype(int)
    melted_df = pd.melt(df, id_vars=['ts','model','state'], value_vars=['time_bet_1_2','time_bet_2_3','time_bet_3_4'],var_name='time_bet', value_name='time')    
    melted_df['color'] = melted_df['model'] + '_' + melted_df['time_bet']
    color_discrete_map = {#"HAMLET-49_time_bet_1_2": 'darkblue',
                          #"HAMLET-49_time_bet_2_3": 'cornflowerblue', 
                          #"HAMLET-49_time_bet_3_4": 'lightblue',
                          "SPT130023W_time_bet_1_2": 'darkblue',
                          "SPT130023W_time_bet_2_3": 'dodgerblue', 
                          "SPT130023W_time_bet_3_4": 'lightskyblue',

                          "SPT120018W_time_bet_1_2": 'maroon',
                          "SPT120018W_time_bet_2_3":'red',
                          "SPT120018W_time_bet_3_4": 'lightcoral',

                          "ORIONT-40_time_bet_1_2": 'forestgreen',
                          "ORIONT-40_time_bet_2_3": 'limegreen', 
                          "ORIONT-40_time_bet_3_4": 'lightgreen',
                          
                          "OCTAN-40_time_bet_1_2": 'indigo',
                          "OCTAN-40_time_bet_2_3": 'mediumorchid', 
                          "OCTAN-40_time_bet_3_4": 'plum',}
    
    #display(melted_df)
    melted_df['time'] = round(melted_df['time']/60,2)
    fig = px.bar(melted_df, x="ts", y="time", 
                labels={"ts": "Timestamp", "time": "Time (minutes)"},
                color = 'color',
                color_discrete_map = color_discrete_map,
                title="Time Between Stations")
    fig.show()






df_bet_all = all_time_between_station(df_bet_12, df_bet_23,df_bet_34)
#display(df_bet_all)
perc_all = perc_of_gabarits_found(df_bet_all, df4, 4)

plot_time_in_station(df_bet_all)
    

Percentage of gabarits scanned in every sensor: 10.21%, which of those occured OK:33 and KO:1


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [94]:
def remove_outliers(df, av_time):
    # Ensure the column 'time_in_station' exists
    if 'time_in_station' not in df.columns:
        print("Error: 'time_in_station' column not found.")
        return df
    

    # Remove rows where 'time_in_station' is greater than 5 minutes
    df = df[df['time_in_station'] <= av_time / 60]

    return df

    
def remove_outlier_quart(df,value):
    # Calculate the first quartile (Q1) and third quartile (Q3)
    filtered_df = pd.DataFrame
    Q1 = df[value].quantile(0.25)
    Q3 = df[value].quantile(0.75)

    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds to filter outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out values outside the specified range
    filtered_df = df[(df[value] >= lower_bound) & (df[value] <= upper_bound)]

    return(filtered_df)

In [95]:
def avg_time_fun(df):
    # Group by 'model' and calculate the median for each group in df_med
    df_avg = pd.DataFrame(columns = ['date','model','time_bet_1_2', 'time_bet_2_3', 'time_bet_3_4','total_time'])

    df['ts'] = pd.to_datetime(df['ts'])
    date = df['ts'].dt.date

    df['total_time'] = df[['time_bet_1_2', 'time_bet_2_3', 'time_bet_3_4']].sum(axis=1)
    df_plot = df
    display(df)

    df = df.drop(['ts', 'gabarit','state'], axis=1) #doesn't matter
  
    avg_total_time = df.groupby('model').mean()
    df_avg = avg_total_time.reset_index()
    df_avg['date'] = date
        
    #display(df_med)

    return df_plot,df_avg #, median_total_time_s


def plot_ok_ko(df):
        #display(melted_df)
    color_discrete_map = {"OK": 'limegreen',
                          "KO": 'red'}
    #display(df)
    df.loc[:,'total_time'] = round(df['total_time']/60,2)
    fig = px.bar(df, x="ts", y='total_time', 
                labels={"ts": "Timestamp", 'total_time': "Time (minutes)"},
                #
                color = 'state',
                #base = 'ts',
                color_discrete_map = color_discrete_map,
                #base = 'state',
                #width=10, # customize width here
                #argap = 0.5,
                title="Average Total Time (morning)",)
    target_date = pd.to_datetime(f"2023-{month}-{day}")

    fig.update_xaxes(range=[target_date + pd.to_timedelta("09:00:00"), target_date + pd.to_timedelta("13:00:00")])
  # Set your desired date range
    fig.show()

    fig2 = px.bar(df, x="ts", y='total_time', 
                labels={"ts": "Timestamp", 'total_time': "Time (minutes)"},
                color = 'state',
                color_discrete_map = color_discrete_map,
                #width=10, # customize width here
                #argap = 0.5,
                title="Average Total Time (afternoon)")
    target_date = pd.to_datetime(f"2023-{month}-{day}")

    fig2.update_xaxes(range=[target_date + pd.to_timedelta("14:00:00"), target_date + pd.to_timedelta("18:00:00")])
  # Set your desired date range
    fig2.show()


#median_time,median_total_time = 
df_sum_time, df_med = avg_time_fun(df_bet_all)
df_sum_time_filt = remove_outlier_quart(df_sum_time, 'total_time')
plot_ok_ko(df_sum_time_filt)



,ts,gabarit,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,state,total_time
0,2023-09-27 09:05:20,15,SPT120018W,41,233,57,OK,331
1,2023-09-27 09:15:49,22,SPT120018W,3,251,201,OK,455
2,2023-09-27 09:17:19,17,SPT120018W,5,299,61,OK,365
3,2023-09-27 09:20:40,15,SPT120018W,4,203,24,OK,231
4,2023-09-27 09:22:11,5,SPT120018W,2,196,119,OK,317
5,2023-09-27 09:25:42,24,SPT120018W,3,298,40,OK,341
6,2023-09-27 09:28:19,3,SPT120018W,3,293,126,OK,422
7,2023-09-27 09:29:33,19,SPT120018W,4,273,71,OK,348
8,2023-09-27 09:42:28,10,SPT120018W,4,231,157,OK,392
9,2023-09-27 09:45:24,19,SPT120018W,2,244,83,OK,329


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



This section will now gather a range of days and calculate the time in and between stations and then present the average per day.


In [96]:
start_date = '25/09/2023'
end_date = '11/10/2023'

start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
end_date = pd.to_datetime(end_date, format='%d/%m/%Y')

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

df_all_med = pd.DataFrame()

def aggregate_dates(df_all_med,date_range):
    
    print_v = 0
    for date in date_range:

        day = str(date.day).zfill(2)
        month = str(date.month).zfill(2)
        date_ = day + "_" + month
        print(date_)

        df1 = process_file("1",date_)
        if df1.empty:
            print("No data in that date")
            continue            
        df2 = process_file("2",date_)
        df3 = process_file("3",date_)
        df4 = process_file("final",date_)
        if df4.empty:
            print("No testing data in that date")
            continue     
        if print_v:
            print(df1)
            print(df2)
            print(df3)
            print(df4)


        df_in_1 = time_in_station_fun(df1)
        df_in_2 = time_in_station_fun(df2)
        df_in_3 = time_in_station_fun(df3)
        if print_v:
            print(df_in_1)
            print(df_in_2)
            print(df_in_3)


        df_bet_12 = time_between_station(df_in_1, df_in_2, 1, 2, max_time = 5)
        df_bet_23 = time_between_station(df_in_2, df_in_3, 2, 3, max_time = 20)
        df_bet_34 = time_between_station(df_in_3, df4, 3, 4, max_time = 0)
        if print_v:
            print(df_bet_12)
            print(df_bet_23)
            print(df_bet_34)


        perc1 = perc_of_gabarits_found(df_in_1, df4, 1)
        perc2 = perc_of_gabarits_found(df_in_2, df4, 2)
        perc3 = perc_of_gabarits_found(df_in_3, df4, 3)

        df_bet_all = all_time_between_station(df_bet_12, df_bet_23,df_bet_34)
        perc4 = perc_of_gabarits_found(df_bet_all, df4, 4)
        if print_v:
            print(df_bet_all)
        
        _,df_med = avg_time_fun(df_bet_all)
        display(df_med)
        print()

        df_all_med = pd.concat([df_all_med, df_med])


    return df_all_med

df_all_med = aggregate_dates(df_all_med,date_range)





25_09
Percentage of gabarits scanned in sensor 1: 30.92%
Percentage of gabarits scanned in sensor 2: 64.47%
Percentage of gabarits scanned in sensor 3: 45.39%
Percentage of gabarits scanned in every sensor: 11.18%, which of those occured OK:33 and KO:1


,ts,gabarit,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,state,total_time
0,2023-09-25 10:15:26,3,SPT120018W,29.0,155.0,91.0,OK,275.0
1,2023-09-25 10:16:52,17,SPT120018W,12.0,206.0,111.0,OK,329.0
2,2023-09-25 10:18:18,9,SPT120018W,3.0,253.0,93.0,OK,349.0
3,2023-09-25 10:20:25,24,SPT120018W,81.0,305.0,202.0,OK,588.0
4,2023-09-25 10:51:09,3,SPT120018W,5.0,194.0,213.0,OK,412.0
5,2023-09-25 10:53:57,9,SPT120018W,41.0,278.0,61.0,OK,380.0
6,2023-09-25 10:58:22,24,SPT120018W,56.0,181.0,67.0,OK,304.0
7,2023-09-25 11:10:51,3,SPT120018W,5.0,197.0,46.0,KO,248.0
8,2023-09-25 11:12:06,17,SPT120018W,5.0,267.0,95.0,OK,367.0
9,2023-09-25 11:17:13,24,SPT120018W,2.0,206.0,151.0,OK,359.0


,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,total_time,date
0,SPT120018W,30.147059,308.294118,83.558824,422.0,2023-09-25



26_09
No data in that date
27_09
Percentage of gabarits scanned in sensor 1: 80.78%
Percentage of gabarits scanned in sensor 2: 22.82%
Percentage of gabarits scanned in sensor 3: 48.05%
Percentage of gabarits scanned in every sensor: 10.21%, which of those occured OK:33 and KO:1


,ts,gabarit,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,state,total_time
0,2023-09-27 09:05:20,15,SPT120018W,41.0,233.0,57.0,OK,331.0
1,2023-09-27 09:15:49,22,SPT120018W,3.0,251.0,201.0,OK,455.0
2,2023-09-27 09:17:19,17,SPT120018W,5.0,299.0,61.0,OK,365.0
3,2023-09-27 09:20:40,15,SPT120018W,4.0,203.0,24.0,OK,231.0
4,2023-09-27 09:22:11,5,SPT120018W,2.0,196.0,119.0,OK,317.0
5,2023-09-27 09:25:42,24,SPT120018W,3.0,298.0,40.0,OK,341.0
6,2023-09-27 09:28:19,3,SPT120018W,3.0,293.0,126.0,OK,422.0
7,2023-09-27 09:29:33,19,SPT120018W,4.0,273.0,71.0,OK,348.0
8,2023-09-27 09:42:28,10,SPT120018W,4.0,231.0,157.0,OK,392.0
9,2023-09-27 09:45:24,19,SPT120018W,2.0,244.0,83.0,OK,329.0


...

,ts,gabarit,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,state,total_time
0,2023-10-10 09:23:51,3,SPT130023W,3.0,511.0,125.0,OK,639.0
1,2023-10-10 09:39:10,10,SPT130023W,36.0,196.0,113.0,OK,345.0
2,2023-10-10 09:41:55,3,SPT130023W,32.0,168.0,122.0,OK,322.0
3,2023-10-10 09:43:09,12,SPT130023W,55.0,122.0,71.0,OK,248.0
4,2023-10-10 09:49:08,5,SPT130023W,3.0,216.0,90.0,OK,309.0
5,2023-10-10 09:56:30,1,SPT130023W,4.0,213.0,93.0,OK,310.0
6,2023-10-10 10:03:23,5,SPT130023W,10.0,221.0,95.0,OK,326.0
7,2023-10-10 10:11:06,1,SPT130023W,57.0,152.0,219.0,OK,428.0
8,2023-10-10 10:14:49,17,SPT130023W,4.0,259.0,100.0,OK,363.0
9,2023-10-10 10:20:11,15,SPT130023W,13.0,251.0,104.0,OK,368.0


,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,total_time,date
0,SPT130023W,38.253731,244.940299,120.0,403.19403,2023-10-10



11_10
Percentage of gabarits scanned in sensor 1: 78.15%
Percentage of gabarits scanned in sensor 2: 27.41%
Percentage of gabarits scanned in sensor 3: 63.33%
Percentage of gabarits scanned in every sensor: 9.63%, which of those occured OK:25 and KO:1


,ts,gabarit,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,state,total_time
0,2023-10-11 09:00:45,3,SPT130023W,79.0,388.0,71.0,OK,538.0
1,2023-10-11 09:04:32,8,SPT130023W,66.0,297.0,90.0,OK,453.0
2,2023-10-11 09:10:53,21,SPT130023W,10.0,251.0,99.0,OK,360.0
3,2023-10-11 09:15:52,17,SPT130023W,63.0,195.0,124.0,OK,382.0
4,2023-10-11 09:19:55,15,SPT130023W,72.0,144.0,210.0,OK,426.0
5,2023-10-11 09:27:34,10,SPT130023W,7.0,228.0,80.0,OK,315.0
6,2023-10-11 09:33:58,5,SPT130023W,114.0,204.0,160.0,OK,478.0
7,2023-10-11 09:48:17,5,SPT130023W,138.0,200.0,75.0,OK,413.0
8,2023-10-11 09:55:46,21,SPT130023W,56.0,278.0,23.0,OK,357.0
9,2023-10-11 10:08:05,12,SPT130023W,76.0,270.0,89.0,OK,435.0


,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,total_time,date
0,SPT130023W,56.115385,277.269231,154.153846,487.538462,2023-10-11


In [97]:

def plot_time_in_station_days(df):
    display(df)
    
    color_discrete_map_ = {"SPT130023W": 'dodgerblue',
                          "SPT120018W": 'red',
                          "ORIONT-40": 'limegreen', 
                          "OCTAN-40": 'mediumorchid'
                          }


    # Create a bar plot for each model
    df['total_time'] = df['total_time']/60
    #df['med_total_time'] = df['med_total_time'].astype(int)
    fig = px.bar(df, x='date', y='total_time', 
                 color='model', 
                 color_discrete_map = color_discrete_map_,
                 barmode='group')

    # Update layout for better visualization

    fig.update_layout(
        title='Median Total Time by Model',
        xaxis_title='Date',
        yaxis_title='Median Total Time (minutes)',
        legend_title='Model',
        barmode='group'
    )

    # Show the plot
    fig.show()

    melted_df = pd.melt(df, id_vars=['date','model','total_time'], value_vars=['time_bet_1_2','time_bet_2_3','time_bet_3_4'],var_name='time_bet', value_name='time')    
    melted_df['color'] = melted_df['model'] + '_' + melted_df['time_bet']
    color_discrete_map = {"SPT130023W_time_bet_1_2": 'darkblue',
                          "SPT130023W_time_bet_2_3": 'dodgerblue', 
                          "SPT130023W_time_bet_3_4": 'lightskyblue',
                          #"HAMLET-49_time_bet_1_2": 'darkblue',

                          "SPT120018W_time_bet_1_2": 'maroon',
                          "SPT120018W_time_bet_2_3":'red',
                          "SPT120018W_time_bet_3_4": 'lightcoral',
                          #"ORIONT-40_time_bet_1_2": 'forestgreen',
                          #"ORIONT-40_time_bet_2_3": 'limegreen', 
                          #"ORIONT-40_time_bet_3_4": 'lightgreen',
                            
                          #"ORIONT-40_time_bet_1_2": 'forestgreen',
                          #"ORIONT-40_time_bet_2_3": 'limegreen', 
                          #"ORIONT-40_time_bet_3_4": 'lightgreen',
                          
                          "OCTAN-40_time_bet_1_2": 'indigo',
                          "OCTAN-40_time_bet_2_3": 'mediumorchid', 
                          "OCTAN-40_time_bet_3_4": 'plum',}



    #model	time_bet_1_2, time_bet_2_3, time_bet_3_4, avg_total_time, date
    
    for model in melted_df['model'].unique():
        df_model = melted_df[melted_df['model'] == model]

        df_model.loc[:,'time_'] = df_model['time']/60
        print("yo")
        #df_model['time'] = df_model['time'].astype(int)
        # Create a bar plot for the current model
        fig4 = px.bar(df_model, x='date', y='time_', 
                    color='color',
                    labels={'time': 'Time'},
                    color_discrete_map = color_discrete_map,
                    title=f'Time per process for Model: {model}')


        #
        fig4.update_xaxes(range=[start_date - pd.Timedelta(days = 1), end_date + pd.Timedelta(days = 1)])  # Set your desired date range


        # Update layout for better visualization
        fig4.update_layout(
            xaxis_title='Date',
            yaxis_title='Time per station (minutes)',
            legend_title='Time Bet',
        )

        # Show the plot
        fig4.show()


plot_time_in_station_days(df_all_med)


,model,time_bet_1_2,time_bet_2_3,time_bet_3_4,total_time,date
0,SPT120018W,30.147059,308.294118,83.558824,422.0,2023-09-25
0,SPT120018W,6.235294,216.147059,92.176471,314.558824,2023-09-27
0,SPT120018W,13.083333,222.166667,73.458333,308.708333,2023-09-28
0,SPT120018W,24.45283,311.377358,85.54717,421.377358,2023-10-02
0,SPT120018W,28.402174,261.141304,144.652174,434.195652,2023-10-03
0,SPT120018W,19.0,296.846154,66.615385,382.461538,2023-10-06
0,SPT120018W,31.185714,270.328571,152.771429,454.285714,2023-10-09
0,SPT130023W,38.253731,244.940299,120.0,403.19403,2023-10-10
0,SPT130023W,56.115385,277.269231,154.153846,487.538462,2023-10-11


yo


C:\Users\isacc\AppData\Local\Temp\ipykernel_7636\3030504899.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



yo


C:\Users\isacc\AppData\Local\Temp\ipykernel_7636\3030504899.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

